In [29]:
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

from __future__ import absolute_import #导入3.x的特征函数
from __future__ import print_function

In [30]:
neg=pd.read_excel('E:/machine_data/text_emotion/neg.xls',header=None,index=None)
pos=pd.read_excel('E:/machine_data/text_emotion/pos.xls',header=None,index=None) #读取训练语料完毕

In [31]:
print (neg.shape)
print (neg[1:4])

(10428, 1)
                                                   0
1  作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...
2  作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
3       作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。


In [32]:
neg.columns

Int64Index([0], dtype='int64')

In [33]:
pos['mark']=1
neg['mark']=0 #给训练语料贴上标签

In [34]:
print (neg.shape)
print (neg[1:4])

(10428, 2)
                                                   0  mark
1  作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...     0
2  作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...     0
3       作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。     0


In [35]:
print (pos.shape)

(10677, 2)


In [36]:
pn=pd.concat([pos,neg],ignore_index=True) #合并语料
neglen=len(neg)
poslen=len(pos) #计算语料数目

In [37]:
print (pn[1:3])

                                                   0  mark
1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...     1
2  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...     1


In [38]:
print (pn.shape)
print (pn[0][1:3])
print (pn['mark'][1:3])

(21105, 2)
1    作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2    作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
Name: 0, dtype: object
1    1
2    1
Name: mark, dtype: int64


In [39]:
cw = lambda x: list(jieba.cut(x)) #定义分词函数
pn['words'] = pn[0].apply(cw)

In [40]:
print (pn[1:3])

                                                   0  mark  \
1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...     1   
2  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...     1   

                                               words  
1  [作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...  
2  [作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ...  


In [41]:
#语料库
comment = pd.read_excel('E:/machine_data/text_emotion/sum.xls') #读入评论内容
#comment = pd.read_csv('a.csv', encoding='utf-8')
comment = comment[comment['rateContent'].notnull()] #仅读取非空评论
comment['words'] = comment['rateContent'].apply(cw) #评论分词 

In [42]:
comment[1:3]

,aliMallSeller,anony,appendComment,attributes,auctionSku,buyCount,carServiceLocation,cmsSource,displayRatePic,displayRateSum,...,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,words
323,False,True,NaN,NaN,颜色分类:白色,0,NaN,天猫,b_red_2.gif,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[还, 没用, 呢, 过, 几天, 在, 追加, 评价]"
324,False,True,NaN,NaN,颜色分类:白色,0,NaN,天猫,b_red_1.gif,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[看, 产品, 很, 好, ，, 用, 起来, 比较, 方便, ，, 给, 好评]"


In [43]:
print (comment['rateContent'][1:3])

323         还没用呢过几天在追加评价
324    看产品很好，用起来比较方便，给好评
Name: rateContent, dtype: object


In [55]:
d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True) 

w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)

In [56]:
print (d2v_train[1:5])
print (w[1:4])

1    [作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...
2    [作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ...
3    [作者, 在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, ...
4    [作者, 在, 少年, 时即, 喜, 阅读, ，, 能, 看出, 他, 精读, 了, 无数,...
Name: words, dtype: object
['父母', '一定', '要']


In [57]:
dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
print (dict[0:3])
del w,d2v_train
dict['id']=list(range(1,len(dict)+1))
print (dict[0:3])

        0
，  131885
的   85960
。   49352
        0  id
，  131885   1
的   85960   2
。   49352   3


In [68]:
get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢

In [69]:
print (dict['id'][0:3])
print (pn['sent'][0:3])

，    1
的    2
。    3
Name: id, dtype: int64
0    [151, 521, 266, 45, 18, 14529, 97, 2, 1787, 1,...
1    [146, 4028, 22742, 2860, 2, 838, 1, 736, 793, ...
2    [146, 7681, 5540, 1045, 4619, 18766, 364, 23, ...
Name: sent, dtype: object


In [70]:
pn['sent'].shape

(21105,)

In [71]:
maxlen = 50 #截取50

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))

Pad sequences (samples x time)


In [74]:
print (pn['sent'][0].shape)
print (pn['sent'][0:3])

(50,)
0    [23, 5254, 872, 1229, 11, 5, 356, 9965, 1, 327...
1    [138, 11327, 4, 1635, 2872, 1, 258, 1364, 2235...
2    [905, 182, 13, 1025, 1119, 50, 1489, 2703, 2, ...
Name: sent, dtype: object


In [80]:
x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

print('Build model...')
model = Sequential()
model.add(Embedding(len(dict)+1, 256))
model.add(LSTM(128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

model.fit(xa, ya, batch_size=16, nb_epoch=10) #训练时间为若干个小时

classes = model.predict_classes(xa)
acc = np_utils.accuracy(classes, ya)
print('Test accuracy:', acc)

Build model...


D:\anaconda1\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
D:\anaconda1\lib\site-packages\keras\backend\tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Epoch 1/10
21105/21105 [==============================] - 420s - loss: 0.3596   
Epoch 2/10
21105/21105 [==============================] - 420s - loss: 0.1438   
Epoch 3/10
21105/21105 [==============================] - 430s - loss: 0.0730   
Epoch 4/10
21105/21105 [==============================] - 413s - loss: 0.0374   
Epoch 5/10
21105/21105 [==============================] - 419s - loss: 0.0295   
Epoch 6/10
21105/21105 [==============================] - 424s - loss: 0.0209   
Epoch 7/10
21105/21105 [==============================] - 420s - loss: 0.0132   
Epoch 8/10
21105/21105 [==============================] - 419s - loss: 0.0121   
Epoch 9/10
21105/21105 [==============================] - 420s - loss: 0.0068   
Epoch 10/10
21056/21105 [============================>.] - ETA: 0s

AttributeError: module 'keras.utils.np_utils' has no attribute 'accuracy'

In [75]:
a=[1,2,3,4,5,6]
a[::2]

[1, 3, 5]

In [76]:
a[1::2]

[2, 4, 6]

In [83]:
score = model.evaluate(xa, ya)

21105/21105 [==============================] - 15s    


In [84]:
print (score)

0.00597528860014


In [85]:
print (classes)
print (ya)

[[1]
 [1]
 [1]
 ..., 
 [0]
 [0]
 [0]]
[1 1 1 ..., 0 0 0]


In [88]:
c=0
for i in range(len(ya)):
    if classes[i]==ya[i]:
        c=c+1
score=c/len(ya)

In [90]:
print(score)
print (len(ya))
print(c)

0.9987206823027719
21105
21078
